# Retirement Trends
### Get:
* Rolling Monthly Retirement and Issuance (and balance)
* What is being retired

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots

from data_functions_copy import Retrieve_Data
app = Retrieve_Data()

### PLOTTING ROLLING ISSUANCES AND RETIREMENTS
#### Seasonality

In [94]:
df_issuance = app.df_issuance.copy().drop_duplicates()
df_issuance['Issuance Date'] = pd.to_datetime(df_issuance['Issuance Date'])

df_retirement = app.df_retirement.copy().drop_duplicates()
df_retirement['Date of Retirement'] = pd.to_datetime(df_retirement['Date of Retirement'])

In [95]:
# tenors = ['Week','Month','Year']
def dated_grouping(df_iss, df_ret, tenor='Month'):
    sub_iss = df_iss.copy()
    sub_iss['Year'] = [i.year for i in sub_iss['Issuance Date']]
    sub_ret = df_ret.copy()
    sub_ret['Year'] = [i.year for i in sub_ret['Date of Retirement']]
    
    if tenor=='Year':
        sub_iss = sub_iss.groupby(by='Year').sum()['Quantity of Units Issued'].reset_index()
        sub_ret = sub_ret.groupby(by='Year').sum()['Quantity of Units'].reset_index()
    elif tenor=='Month':
        sub_iss[tenor] = [i.month for i in sub_iss['Issuance Date']]
        sub_iss = sub_iss.groupby(by=['Year',tenor]).sum()['Quantity of Units Issued'].reset_index()
        sub_ret[tenor] = [i.month for i in sub_ret['Date of Retirement']]
        sub_ret = sub_ret.groupby(by=['Year',tenor]).sum()['Quantity of Units'].reset_index()
    elif tenor=='Week':
        sub_iss[tenor] = [i.week for i in sub_iss['Issuance Date']]
        sub_iss = sub_iss.groupby(by=['Year',tenor]).sum()['Quantity of Units Issued'].reset_index()
        sub_ret[tenor] = [i.week for i in sub_ret['Date of Retirement']]
        sub_ret = sub_ret.groupby(by=['Year',tenor]).sum()['Quantity of Units'].reset_index()        
    return sub_iss, sub_ret

In [112]:
#iss, ret = dated_grouping(df_issuance, df_retirement, tenor='Year')
def monthly_plots(df_iss, df_ret, min_year='None', tenor='Month'):
    sub_iss, sub_ret = dated_grouping(df_iss, df_ret, tenor)
    #sub_ret = df_ret.copy()
    if min_year=='None':
        pass
    else:
        sub_ret = sub_ret[sub_ret.Year>=min_year].reset_index(drop=True)
        sub_iss = sub_iss[sub_iss.Year>=min_year].reset_index(drop=True)
    fig_iss = px.bar(sub_iss, x=tenor,y='Quantity of Units Issued',color='Year')
    fig_ret = px.bar(sub_ret, x=tenor,y='Quantity of Units',color='Year')
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("{}ly Issuances".format(tenor),"{}ly Retirements".format(tenor)))
    fig.add_trace(fig_iss.data[0], row=1, col=1)
    fig.add_trace(fig_ret.data[0], row=1, col=2)
    fig.show()
    
monthly_plots(df_issuance, df_retirement, min_year='None', tenor='Year')

In [113]:
monthly_plots(df_issuance, df_retirement, min_year=2020, tenor='Month')

### Rolling

#### Rolling - Monthly

In [120]:
monthly_issuance = df_issuance.groupby(pd.Grouper(key='Issuance Date', freq='M')).sum()['Quantity of Units Issued'].reset_index()
monthly_retirement = df_retirement.groupby(pd.Grouper(key='Date of Retirement', freq='M')).sum()['Quantity of Units'].reset_index()

In [121]:
px.bar(monthly_retirement, x='Date of Retirement',y='Quantity of Units').show()

In [122]:
px.bar(monthly_issuance, x='Issuance Date',y='Quantity of Units Issued').show()

In [115]:
### USE THIS CODE SNIPPET EXAMPLE TO CREATE A STACKED BAR CHART WITH RETIREMENTS AS NEGATIVES ###
# Is there a way for another linked chart below that to be the spread?


import plotly.graph_objects as go

# Sample data
categories = ['Category 1', 'Category 2', 'Category 3']
positive_values = [10, 20, 30]
negative_values = [-5, -15, -10]

# Create the trace for positive values
trace_positive = go.Bar(
    x=categories,
    y=positive_values,
    name='Positive',
    marker_color='blue'
)

# Create the trace for negative values
trace_negative = go.Bar(
    x=categories,
    y=negative_values,
    name='Negative',
    marker_color='red'
)

# Create the stacked bar chart
fig = go.Figure(data=[trace_positive, trace_negative])
fig.update_layout(barmode='relative')

# Display the chart
fig.show()


,Issuance Date,Quantity of Units Issued
0,2009-03-22,39064
1,2009-03-29,786
2,2009-04-05,1566734
3,2009-04-12,0
4,2009-04-19,64951
...,...,...
739,2023-05-21,1746684
740,2023-05-28,7141537
741,2023-06-04,3648804
742,2023-06-11,2652295


In [84]:
x.reset_index()

,Issuance Date,Quantity of Units Issued
0,2009-03-22,39064
1,2009-03-29,786
2,2009-04-05,1566734
3,2009-04-12,0
4,2009-04-19,64951
...,...,...
739,2023-05-21,1746684
740,2023-05-28,7141537
741,2023-06-04,3648804
742,2023-06-11,2652295


In [91]:
# Frequency can be ['W','M','Y']
def monthly_plots(df_iss, df_ret, min_year='None', frequency='W'):
    sub_iss = df_iss.copy()
    sub_iss['Year'] = [i.year for i in sub_iss['Issuance Date']]
    sub_ret = df_ret.copy()
    sub_ret['Year'] = [i.year for i in sub_ret['Date of Retirement']]
    if min_year=='None':
        pass
    else:
        sub_ret = sub_ret[sub_ret.Year>=min_year].reset_index(drop=True)
        sub_iss = sub_iss[sub_iss.Year>=min_year].reset_index(drop=True)
        
    # Group on the frequency
    sub_iss = sub_iss.groupby(pd.Grouper(key='Issuance Date', freq=frequency)).sum()['Quantity of Units Issued'].reset_index()
    print(sub_iss)
    sub_ret = sub_ret.groupby(pd.Grouper(key='Date of Retirement', freq=frequency)).sum()['Quantity of Units'].reset_index()
    
    fig_iss = px.bar(sub_iss, x=frequency,y='Quantity of Units Issued',color='Year')
    fig_ret = px.bar(sub_ret, x=frequency,y='Quantity of Units',color='Year')
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Monthly Issuances","Monthly Retirements"))
    fig.add_trace(fig_iss.data[0], row=1, col=1)
    fig.add_trace(fig_ret.data[0], row=1, col=2)
    fig.show()
    
monthly_plots(df_issuance, df_retirement, min_year=2019, frequency='M')    

   Issuance Date  Quantity of Units Issued
0     2019-01-31                  23787240
1     2019-02-28                   2940066
2     2019-03-31                   4147305
3     2019-04-30                   4633302
4     2019-05-31                   8890433
5     2019-06-30                   5001635
6     2019-07-31                  13504040
7     2019-08-31                   6489689
8     2019-09-30                   8331377
9     2019-10-31                   7284735
10    2019-11-30                  11504148
11    2019-12-31                  22294864
12    2020-01-31                  11160677
13    2020-02-29                  11665023
14    2020-03-31                  20093610
15    2020-04-30                   3728329
16    2020-05-31                   5344039
17    2020-06-30                   6814273
18    2020-07-31                  15514019
19    2020-08-31                   5805667
20    2020-09-30                   9718572
21    2020-10-31                  10761432
22    2020-

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Issuance Date', 'Quantity of Units Issued'] but received: M

In [92]:
def monthly_plots(df_iss, df_ret, min_year='None'):
    sub_iss = df_iss.copy()
    sub_ret = df_ret.copy()
    if min_year=='None':
        pass
    else:
        sub_ret = df_ret[df_ret.Year>=min_year].reset_index(drop=True)
        sub_iss = df_iss[df_iss.Year>=min_year].reset_index(drop=True)
    fig_iss = px.bar(sub_iss, x='Month',y='Quantity of Units Issued',color='Year')
    fig_ret = px.bar(sub_ret, x='Month',y='Quantity of Units',color='Year')
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Monthly Issuances","Monthly Retirements"))
    fig.add_trace(fig_iss.data[0], row=1, col=1)
    fig.add_trace(fig_ret.data[0], row=1, col=2)
    fig.show()
    
monthly_plots(df_issuance, df_retirement, min_year=2019)    

AttributeError: 'DataFrame' object has no attribute 'Year'

In [48]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Monthly Issuances","Monthly Retirements"))
fig.add_trace(fig_iss.data[0], row=1, col=1)
fig.add_trace(fig_ret.data[0], row=1, col=2)
fig.show()

In [ ]:
app.vintage_retirements()

In [ ]:
app.retirement_ratios()

In [3]:
app.all_project_balances()

,Vintage,ID,Method,Name,Country,Issued,Retired,Balance
0,2000,163,Waste,Imudia Utilization of Coal-Mine-Methane,Germany (DE),2000,2000.0,0.0
1,2000,220,Fugitive Emiss.,"Her Mont, Utilization of Coal-Mine-Methane",Germany (DE),10167,10167.0,0.0
2,2001,57,Waste,Waste Biomass Utilization at JSC Arkhangelsk P...,Russian Federation (RU),115,12.0,103.0
3,2001,93,Hydro,86 MW Hydro Project in Himachal Pradesh,India (IN),122509,122509.0,0.0
4,2001,163,Waste,Imudia Utilization of Coal-Mine-Methane,Germany (DE),59034,59034.0,0.0
...,...,...,...,...,...,...,...,...
7980,2022,3545,Solar,"50 MW (AC) SOLAR PARK AT SUTIAKHALI, GOURIPUR,...",Bangladesh (BD),36116,0.0,36116.0
7981,2022,3649,LFG,Tianjin Dahanzhuang LFG Power Generation Proje...,China (CN),43083,0.0,43083.0
7982,2022,3662,Hydro,15 MW Nam Hinboun Downstream Hydropower Project,Lao (LA),23141,0.0,23141.0
7983,2022,3696,LFG,Chongqing Tongliang LFG Power Generation Project,China (CN),42672,0.0,42672.0


In [ ]:
app.ngeo_project_balances()